In [12]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
loans_df = pd.read_csv('loan_status.csv')
loans_df.head()

,Loan_Status,Current_Loan_Amount,Term,Credit_Score,Annual_Income,Years_in_current_job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens
0,Fully_Paid,99999999,Short_Term,741.0,2231892.0,8_years,Own_Home,Debt_Consolidation,29200.53,14.9,29.0,18,1,297996,750090.0,0.0,0.0
1,Fully_Paid,217646,Short_Term,730.0,1184194.0,<_1_year,Home_Mortgage,Debt_Consolidation,10855.08,19.6,10.0,13,1,122170,272052.0,1.0,0.0
2,Fully_Paid,548746,Short_Term,678.0,2559110.0,2_years,Rent,Debt_Consolidation,18660.28,22.6,33.0,4,0,437171,555038.0,0.0,0.0
3,Fully_Paid,99999999,Short_Term,728.0,714628.0,3_years,Rent,Debt_Consolidation,11851.06,16.0,76.0,16,0,203965,289784.0,0.0,0.0
4,Fully_Paid,99999999,Short_Term,740.0,776188.0,<_1_year,Own_Home,Debt_Consolidation,11578.22,8.5,25.0,6,0,134083,220220.0,0.0,0.0


In [13]:
# Generate our categorical variable list
loans_cat = loans_df.dtypes[loans_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
loans_df[loans_cat].nunique()

Loan_Status              2
Term                     2
Years_in_current_job    11
Home_Ownership           4
Purpose                  7
dtype: int64

In [14]:
# Check the unique value counts to see if binning is required
loans_df.Years_in_current_job.value_counts()

10+_years    13149
2_years       3225
3_years       2997
<_1_year      2699
5_years       2487
4_years       2286
1_year        2247
6_years       2109
7_years       2082
8_years       1675
9_years       1467
Name: Years_in_current_job, dtype: int64

In [15]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(loans_df[loans_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names_out(loans_cat)
encode_df.head()

,Loan_Status_Fully_Paid,Loan_Status_Not_Paid,Term_Long_Term,Term_Short_Term,Years_in_current_job_10+_years,Years_in_current_job_1_year,Years_in_current_job_2_years,Years_in_current_job_3_years,Years_in_current_job_4_years,Years_in_current_job_5_years,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [16]:
# Merge one-hot encoded features and drop the originals
loans_df = loans_df.merge(encode_df,left_index=True, right_index=True)
loans_df = loans_df.drop(loans_cat,1)
loans_df.head()

C:\Users\jckar\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Current_Loan_Amount,Credit_Score,Annual_Income,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,99999999,741.0,2231892.0,29200.53,14.9,29.0,18,1,297996,750090.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,217646,730.0,1184194.0,10855.08,19.6,10.0,13,1,122170,272052.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,548746,678.0,2559110.0,18660.28,22.6,33.0,4,0,437171,555038.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,99999999,728.0,714628.0,11851.06,16.0,76.0,16,0,203965,289784.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,99999999,740.0,776188.0,11578.22,8.5,25.0,6,0,134083,220220.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [17]:
# Remove loan status target from features data
y = loans_df.Loan_Status_Fully_Paid
X = loans_df.drop(columns=["Loan_Status_Fully_Paid","Loan_Status_Not_Paid"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
854/854 [==============================] - 3s 3ms/step - loss: 0.4248 - accuracy: 0.8333
Epoch 2/50
854/854 [==============================] - 2s 3ms/step - loss: 0.3868 - accuracy: 0.8492
Epoch 3/50
854/854 [==============================] - 2s 3ms/step - loss: 0.3830 - accuracy: 0.8494
Epoch 4/50
854/854 [==============================] - 2s 3ms/step - loss: 0.3809 - accuracy: 0.8493
Epoch 5/50
854/854 [==============================] - 2s 3ms/step - loss: 0.3790 - accuracy: 0.8494
Epoch 6/50
854/854 [==============================] - 2s 3ms/step - loss: 0.3782 - accuracy: 0.8497
Epoch 7/50
854/854 [==============================] - 2s 3ms/step - loss: 0.3771 - accuracy: 0.8498
Epoch 8/50
854/854 [==============================] - 2s 3ms/step - loss: 0.3761 - accuracy: 0.8500
Epoch 9/50
854/854 [==============================] - 2s 3ms/step - loss: 0.3755 - accuracy: 0.8500
Epoch 10/50
854/854 [==============================] - 2s 3ms/step - loss: 0.3744 - accuracy: 0.8499

In [19]:
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 24)                888       
                                                                 
 dense_7 (Dense)             (None, 12)                300       
                                                                 
 dense_8 (Dense)             (None, 1)                 13        
                                                                 
Total params: 1,201
Trainable params: 1,201
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [21]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
820/854 [===========================>..] - ETA: 0s - loss: 0.3603 - accuracy: 0.8534
Epoch 00001: saving model to checkpoints\weights.01.hdf5
854/854 [==============================] - 3s 2ms/step - loss: 0.3599 - accuracy: 0.8536
Epoch 2/100
844/854 [============================>.] - ETA: 0s - loss: 0.3595 - accuracy: 0.8530
Epoch 00002: saving model to checkpoints\weights.02.hdf5
854/854 [==============================] - 2s 2ms/step - loss: 0.3599 - accuracy: 0.8529
Epoch 3/100
835/854 [============================>.] - ETA: 0s - loss: 0.3592 - accuracy: 0.8536
Epoch 00003: saving model to checkpoints\weights.03.hdf5
854/854 [==============================] - 2s 2ms/step - loss: 0.3593 - accuracy: 0.8537
Epoch 4/100
852/854 [============================>.] - ETA: 0s - loss: 0.3589 - accuracy: 0.8541
Epoch 00004: saving model to checkpoints\weights.04.hdf5
854/854 [==============================] - 2s 3ms/step - loss: 0.3588 - accuracy: 0.8541
Epoch 5/100
837/854 [=======

854/854 [==============================] - 2s 3ms/step - loss: 0.3541 - accuracy: 0.8557
Epoch 35/100
849/854 [============================>.] - ETA: 0s - loss: 0.3537 - accuracy: 0.8561
Epoch 00035: saving model to checkpoints\weights.35.hdf5
854/854 [==============================] - 2s 3ms/step - loss: 0.3538 - accuracy: 0.8561
Epoch 36/100
836/854 [============================>.] - ETA: 0s - loss: 0.3543 - accuracy: 0.8560
Epoch 00036: saving model to checkpoints\weights.36.hdf5
854/854 [==============================] - 2s 3ms/step - loss: 0.3535 - accuracy: 0.8562
Epoch 37/100
849/854 [============================>.] - ETA: 0s - loss: 0.3535 - accuracy: 0.8554
Epoch 00037: saving model to checkpoints\weights.37.hdf5
854/854 [==============================] - 2s 3ms/step - loss: 0.3535 - accuracy: 0.8554
Epoch 38/100
847/854 [============================>.] - ETA: 0s - loss: 0.3537 - accuracy: 0.8551
Epoch 00038: saving model to checkpoints\weights.38.hdf5
854/854 [===============

849/854 [============================>.] - ETA: 0s - loss: 0.3500 - accuracy: 0.8564
Epoch 00068: saving model to checkpoints\weights.68.hdf5
854/854 [==============================] - 2s 3ms/step - loss: 0.3504 - accuracy: 0.8563
Epoch 69/100
849/854 [============================>.] - ETA: 0s - loss: 0.3510 - accuracy: 0.8560
Epoch 00069: saving model to checkpoints\weights.69.hdf5
854/854 [==============================] - 2s 3ms/step - loss: 0.3509 - accuracy: 0.8561
Epoch 70/100
847/854 [============================>.] - ETA: 0s - loss: 0.3503 - accuracy: 0.8560
Epoch 00070: saving model to checkpoints\weights.70.hdf5
854/854 [==============================] - 2s 3ms/step - loss: 0.3504 - accuracy: 0.8560
Epoch 71/100
845/854 [============================>.] - ETA: 0s - loss: 0.3511 - accuracy: 0.8559
Epoch 00071: saving model to checkpoints\weights.71.hdf5
854/854 [==============================] - 2s 3ms/step - loss: 0.3511 - accuracy: 0.8558
Epoch 72/100
847/854 [===============

# Using the load_weights method

In [24]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

ValueError: Cannot assign value to variable ' dense_15/kernel:0': Shape mismatch.The variable shape (36, 8), and the assigned value shape (36, 24) are incompatible.

# MODULE 19.6.2

In [25]:
# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

In [26]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

In [27]:
# Evaluate the completed model using the test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

285/285 - 1s - loss: 0.6718 - accuracy: 0.5099 - 721ms/epoch - 3ms/step
Loss: 0.6718077063560486, Accuracy: 0.5098835825920105
